In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from difflib import SequenceMatcher

# Functions

In [17]:
#################
##PREPROCESSING##
#################

# Function that preprocess the columns for balance, income and cash flow to create the initial columns 
# like Date, Year, Month, Level 0
def preprocess_columns(df,  name_statement = "Income Statement", nulls= 40):
    
    for col in df.columns:
        if df[col].isnull().sum()>nulls:
            df.drop(columns=col, inplace=True)
    
    if df.index.dtype == 'int64' and "level_0" in df.columns:
        df.drop(columns="level_0", inplace=True)
    

    

    print("Se hace")
    print(df.columns)
    df_extended= df.reset_index()
    print(df_extended.columns)
    
    if df_extended.index.dtype == 'int64' and "level_0" in df_extended.columns:
        df_extended.drop(columns="level_0", inplace=True)

    df_extended = df_extended[::-1].reset_index(drop=True)
    print(df_extended.index)
    df_extended['Level 2 Order'] = df_extended.index+1


    print(f"Cogemos {df_extended.columns[0]}")
    df_extended = pd.melt(
        df_extended,
        id_vars={df_extended.columns[0], "Level 2 Order"},
        var_name= "Date",
        value_name="Value"
    )

    df_extended['Date'] = pd.to_datetime(df_extended['Date'], format ="yyyy-mm-dd")
    df_extended['Year'] = df_extended['Date'].dt.year
    df_extended['Month'] = df_extended['Date'].dt.month

    df_extended['Level 0'] = name_statement

    return df_extended



### Create level 1 order for balances


def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()



total = ["Current Assets", "Total Non Current Assets", "Total Assets", "Current Liabilities",
          "Total Non Current Liabilities Net Minority Interest", "Total Liabilities Net Minority Interest",
            ]

def create_level(df, levels, name_levels, level_str= "Level 1", total= total, threshold=0.95):

    assert(level_str == "Level 1" or level_str == "Level 2")

    level_str_order = level_str +' Order'
    df[level_str] = None
    df[level_str_order] = None

    in_level2 = False
    if level_str == "Level 2": # La columna Highlight solo se creará cuando se cree el nivel 2
        in_level2 = True
        df["Highlight"] = 0

    len_levels = len(levels)

    pos=0
    level = levels[pos]
    name_level = name_levels[pos]
    for idx, row in df.iterrows():
        
        medida = row['index']
        df.at[idx, level_str] = name_level
        df.at[idx, level_str_order] = pos+1

        # Se puede mejorar porque la técnica de NLP es pobre y a veces hay que cambiar el nombre porque los balances cambian
        if similarity(medida, level) >= threshold:
            pos+=1
            if pos == len_levels:
                pos = pos % (len_levels-1) -1 # Se puede mejorar pero sale
            level = levels[pos]
            name_level = name_levels[pos]
            
            if in_level2 and df.at[idx, "index"] not in total: # Solo creamos Highlight cuando estemos en el nivel 2
                df.at[idx, "Highlight"] = 1 # Hay que añadirlo para resaltar el resultado principal (De COGS solo queremos resltar COGS y no lo que haya dentro) 
            print(f"Pasamos a {level}")
            

    return df

# Basic Info

In [5]:
company_name = "Grab"
company = yf.Ticker("GRAB")

# Balance 

In [6]:
balance = company.balance_sheet
balance

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,NaN,0.0,NaN,NaN,NaN
Ordinary Shares Number,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Share Issued,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Total Debt,364000000.0,793000000.0,1365000000.0,2175000000.0,NaN
Tangible Book Value,5424000000.0,5533000000.0,5699000000.0,7058000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,5629000000.0,5043000000.0,5086000000.0,8078000000.0,NaN
Other Short Term Investments,2665000000.0,1905000000.0,3134000000.0,3240000000.0,NaN
Cash And Cash Equivalents,2964000000.0,3138000000.0,1952000000.0,4838000000.0,NaN
Cash Equivalents,861000000.0,650000000.0,504000000.0,NaN,NaN


In [7]:
balance = company.balance_sheet
balance

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,NaN,0.0,NaN,NaN,NaN
Ordinary Shares Number,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Share Issued,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Total Debt,364000000.0,793000000.0,1365000000.0,2175000000.0,NaN
Tangible Book Value,5424000000.0,5533000000.0,5699000000.0,7058000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,5629000000.0,5043000000.0,5086000000.0,8078000000.0,NaN
Other Short Term Investments,2665000000.0,1905000000.0,3134000000.0,3240000000.0,NaN
Cash And Cash Equivalents,2964000000.0,3138000000.0,1952000000.0,4838000000.0,NaN
Cash Equivalents,861000000.0,650000000.0,504000000.0,NaN,NaN


In [8]:
quarterly_balance = company.quarterly_balance_sheet
quarterly_balance

,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
Treasury Shares Number,NaN,NaN,NaN,NaN,NaN,0.0
Ordinary Shares Number,4074138069.0,4070297652.0,4027000000.0,3961000000.0,3953000000.0,NaN
Share Issued,4074138069.0,4070297652.0,4027000000.0,3961000000.0,3953000000.0,NaN
Total Debt,385000000.0,364000000.0,328000000.0,295000000.0,298000000.0,NaN
Tangible Book Value,5454000000.0,5424000000.0,5416000000.0,5351000000.0,5364000000.0,NaN
...,...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,5881000000.0,5629000000.0,5654000000.0,4428000000.0,4005000000.0,NaN
Other Short Term Investments,3053000000.0,2665000000.0,2769000000.0,2169000000.0,1892000000.0,NaN
Cash And Cash Equivalents,2828000000.0,2964000000.0,2885000000.0,2259000000.0,2113000000.0,NaN
Cash Equivalents,NaN,861000000.0,NaN,NaN,NaN,650000000.0


In [9]:
quarterly_balance.tail()

,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
Cash Cash Equivalents And Short Term Investments,5881000000.0,5629000000.0,5654000000.0,4428000000.0,4005000000.0,NaN
Other Short Term Investments,3053000000.0,2665000000.0,2769000000.0,2169000000.0,1892000000.0,NaN
Cash And Cash Equivalents,2828000000.0,2964000000.0,2885000000.0,2259000000.0,2113000000.0,NaN
Cash Equivalents,NaN,861000000.0,NaN,NaN,NaN,650000000.0
Cash Financial,NaN,2103000000.0,NaN,NaN,NaN,2488000000.0


## Anual Balance

In [10]:
balance.head()

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,NaN,0.0,NaN,NaN,NaN
Ordinary Shares Number,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Share Issued,4070297652.0,3933743051.0,3840188991.0,3741980208.0,NaN
Total Debt,364000000.0,793000000.0,1365000000.0,2175000000.0,NaN
Tangible Book Value,5424000000.0,5533000000.0,5699000000.0,7058000000.0,NaN


In [11]:
balance_extended  = preprocess_columns(balance, "Annual Balance")
balance_extended.head()

Se hace
DatetimeIndex(['2024-12-31', '2023-12-31', '2022-12-31', '2021-12-31'], dtype='datetime64[ns]', freq=None)
Index([            'index', 2024-12-31 00:00:00, 2023-12-31 00:00:00,
       2022-12-31 00:00:00, 2021-12-31 00:00:00],
      dtype='object')
RangeIndex(start=0, stop=90, step=1)
Cogemos index


,Level 2 Order,index,Date,Value,Year,Month,Level 0
0,1,Cash Financial,2024-12-31,2103000000.0,2024,12,Annual Balance
1,2,Cash Equivalents,2024-12-31,861000000.0,2024,12,Annual Balance
2,3,Cash And Cash Equivalents,2024-12-31,2964000000.0,2024,12,Annual Balance
3,4,Other Short Term Investments,2024-12-31,2665000000.0,2024,12,Annual Balance
4,5,Cash Cash Equivalents And Short Term Investments,2024-12-31,5629000000.0,2024,12,Annual Balance


## Quarterly balance

In [12]:
quarterly_balance.head()

,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
Treasury Shares Number,NaN,NaN,NaN,NaN,NaN,0.0
Ordinary Shares Number,4074138069.0,4070297652.0,4027000000.0,3961000000.0,3953000000.0,NaN
Share Issued,4074138069.0,4070297652.0,4027000000.0,3961000000.0,3953000000.0,NaN
Total Debt,385000000.0,364000000.0,328000000.0,295000000.0,298000000.0,NaN
Tangible Book Value,5454000000.0,5424000000.0,5416000000.0,5351000000.0,5364000000.0,NaN


In [13]:
quarterly_balance_extended = preprocess_columns(quarterly_balance, "Quarter Balance")
quarterly_balance_extended.head()

Se hace
DatetimeIndex(['2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30',
               '2024-03-31'],
              dtype='datetime64[ns]', freq=None)
Index([            'index', 2025-03-31 00:00:00, 2024-12-31 00:00:00,
       2024-09-30 00:00:00, 2024-06-30 00:00:00, 2024-03-31 00:00:00],
      dtype='object')
RangeIndex(start=0, stop=89, step=1)
Cogemos index


,Level 2 Order,index,Date,Value,Year,Month,Level 0
0,1,Cash Financial,2025-03-31,NaN,2025,3,Quarter Balance
1,2,Cash Equivalents,2025-03-31,NaN,2025,3,Quarter Balance
2,3,Cash And Cash Equivalents,2025-03-31,2828000000.0,2025,3,Quarter Balance
3,4,Other Short Term Investments,2025-03-31,3053000000.0,2025,3,Quarter Balance
4,5,Cash Cash Equivalents And Short Term Investments,2025-03-31,5881000000.0,2025,3,Quarter Balance


In [14]:
quarterly_balance_extended

,Level 2 Order,index,Date,Value,Year,Month,Level 0
0,1,Cash Financial,2025-03-31,NaN,2025,3,Quarter Balance
1,2,Cash Equivalents,2025-03-31,NaN,2025,3,Quarter Balance
2,3,Cash And Cash Equivalents,2025-03-31,2828000000.0,2025,3,Quarter Balance
3,4,Other Short Term Investments,2025-03-31,3053000000.0,2025,3,Quarter Balance
4,5,Cash Cash Equivalents And Short Term Investments,2025-03-31,5881000000.0,2025,3,Quarter Balance
...,...,...,...,...,...,...,...
440,85,Tangible Book Value,2024-03-31,5364000000.0,2024,3,Quarter Balance
441,86,Total Debt,2024-03-31,298000000.0,2024,3,Quarter Balance
442,87,Share Issued,2024-03-31,3953000000.0,2024,3,Quarter Balance
443,88,Ordinary Shares Number,2024-03-31,3953000000.0,2024,3,Quarter Balance


## Create Level 1

Habrá que cambiar los nombres de levels al exacto

In [18]:
levels_1_balance =  ['Current Assets', "Total Assets", "Current Liabilities", "Total Liabilities Net Minority Interest", "Treasury Shares Number"]
name_levels_1_balance = ['Current Assets', "Non Current Assets", "Current Liabilities", "Non Current Liabilities", "Equity"]

quarterly_balance_level1 = create_level(quarterly_balance_extended, levels_1_balance, name_levels_1_balance)

Pasamos a Total Assets
Pasamos a Current Liabilities
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Current Assets
Pasamos a Total Assets
Pasamos a Current Liabilities
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Current Assets
Pasamos a Total Assets
Pasamos a Current Liabilities
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Current Assets
Pasamos a Total Assets
Pasamos a Current Liabilities
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Current Assets
Pasamos a Total Assets
Pasamos a Current Liabilities
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Current Assets


In [19]:
quarterly_balance_level1

,Level 2 Order,index,Date,Value,Year,Month,Level 0,Level 1,Level 1 Order
0,1,Cash Financial,2025-03-31,NaN,2025,3,Quarter Balance,Current Assets,1
1,2,Cash Equivalents,2025-03-31,NaN,2025,3,Quarter Balance,Current Assets,1
2,3,Cash And Cash Equivalents,2025-03-31,2828000000.0,2025,3,Quarter Balance,Current Assets,1
3,4,Other Short Term Investments,2025-03-31,3053000000.0,2025,3,Quarter Balance,Current Assets,1
4,5,Cash Cash Equivalents And Short Term Investments,2025-03-31,5881000000.0,2025,3,Quarter Balance,Current Assets,1
...,...,...,...,...,...,...,...,...,...
440,85,Tangible Book Value,2024-03-31,5364000000.0,2024,3,Quarter Balance,Equity,5
441,86,Total Debt,2024-03-31,298000000.0,2024,3,Quarter Balance,Equity,5
442,87,Share Issued,2024-03-31,3953000000.0,2024,3,Quarter Balance,Equity,5
443,88,Ordinary Shares Number,2024-03-31,3953000000.0,2024,3,Quarter Balance,Equity,5


## Create Level 2 and Highlight

In [20]:
total = ["Current Assets", "Total Non Current Assets", "Total Assets", "Current Liabilities",
          "Total Non Current Liabilities Net Minority Interest", "Total Liabilities Net Minority Interest",
            ]

In [21]:
highlighted_items = [
    "Cash Cash Equivalents And Short Term Investments",
    "Receivables",
    "Inventory",
    "Other Current Assets",
    "Current Assets",
    "Net PPE",
    "Goodwill And Other Intangible Assets",
    "Investments And Advances",
    "Non Current Note Receivables",
    "Non Current Deferred Assets",
    "Other Non Current Assets",
    "Total Non Current Assets",
    "Total Assets",
    "Payables And Accrued Expenses",
    "Current Provisions",
    "Current Debt And Capital Lease Obligation",
    "Current Deferred Liabilities",
    "Current Liabilities",
    "Long Term Provisions",
    "Long Term Debt And Capital Lease Obligation",
    "Non Current Deferred Liabilities",
    "Other Non Current Liabilities",
    "Total Non Current Liabilities Net Minority Interest",
    "Total Liabilities Net Minority Interest",
    "Total Equity Gross Minority Interest",
    "Treasury Shares Number"
]
highlighted_items_names = [
    "Cash Cash Equivalents And Short Term Investments",
    "Receivables",
    "Inventory",
    "Other Current Assets",
    "Current Assets",
    "Net PPE",
    "Goodwill And Other Intangible Assets",
    "Investments And Advances",
    "Non Current Note Receivables",
    "Non Current Deferred Assets",
    "Other Non Current Assets",
    "Total Non Current Assets",
    "Total Assets",
    "Payables And Accrued Expenses",
    "Current Provisions",
    "Current Debt And Capital Lease Obligation",
    "Current Deferred Liabilities",
    "Current Liabilities",
    "Long Term Provisions",
    "Long Term Debt And Capital Lease Obligation",
    "Non Current Deferred Liabilities",
    "Other Non Current Liabilities",
    "Total Non Current Liabilities Net Minority Interest",
    "Total Liabilities Net Minority Interest",
    "Total Equity Gross Minority Interest",
    "Other Data"
]


quarterly_balance_level1_2 = create_level(quarterly_balance_extended, highlighted_items, highlighted_items_names, level_str="Level 2")

Pasamos a Receivables
Pasamos a Inventory
Pasamos a Other Current Assets
Pasamos a Current Assets
Pasamos a Net PPE
Pasamos a Goodwill And Other Intangible Assets
Pasamos a Investments And Advances
Pasamos a Non Current Note Receivables
Pasamos a Non Current Deferred Assets
Pasamos a Other Non Current Assets
Pasamos a Total Non Current Assets
Pasamos a Total Assets
Pasamos a Payables And Accrued Expenses
Pasamos a Current Provisions
Pasamos a Current Debt And Capital Lease Obligation
Pasamos a Current Deferred Liabilities
Pasamos a Current Liabilities
Pasamos a Long Term Provisions
Pasamos a Long Term Debt And Capital Lease Obligation
Pasamos a Non Current Deferred Liabilities
Pasamos a Other Non Current Liabilities
Pasamos a Total Non Current Liabilities Net Minority Interest
Pasamos a Total Liabilities Net Minority Interest
Pasamos a Total Equity Gross Minority Interest
Pasamos a Treasury Shares Number
Pasamos a Cash Cash Equivalents And Short Term Investments
Pasamos a Receivables
P

In [22]:
quarterly_balance_level1_2

,Level 2 Order,index,Date,Value,Year,Month,Level 0,Level 1,Level 1 Order,Level 2,Highlight
0,1,Cash Financial,2025-03-31,NaN,2025,3,Quarter Balance,Current Assets,1,Cash Cash Equivalents And Short Term Investments,0
1,1,Cash Equivalents,2025-03-31,NaN,2025,3,Quarter Balance,Current Assets,1,Cash Cash Equivalents And Short Term Investments,0
2,1,Cash And Cash Equivalents,2025-03-31,2828000000.0,2025,3,Quarter Balance,Current Assets,1,Cash Cash Equivalents And Short Term Investments,0
3,1,Other Short Term Investments,2025-03-31,3053000000.0,2025,3,Quarter Balance,Current Assets,1,Cash Cash Equivalents And Short Term Investments,0
4,1,Cash Cash Equivalents And Short Term Investments,2025-03-31,5881000000.0,2025,3,Quarter Balance,Current Assets,1,Cash Cash Equivalents And Short Term Investments,1
...,...,...,...,...,...,...,...,...,...,...,...
440,26,Tangible Book Value,2024-03-31,5364000000.0,2024,3,Quarter Balance,Equity,5,Other Data,0
441,26,Total Debt,2024-03-31,298000000.0,2024,3,Quarter Balance,Equity,5,Other Data,0
442,26,Share Issued,2024-03-31,3953000000.0,2024,3,Quarter Balance,Equity,5,Other Data,0
443,26,Ordinary Shares Number,2024-03-31,3953000000.0,2024,3,Quarter Balance,Equity,5,Other Data,0


# Exportamos balances

In [23]:
balance_extended.to_excel(f"C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/balance.xlsx", sheet_name= "Balance Sheet")
quarterly_balance_level1.to_excel(f"C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/quarterly_balance.xlsx", sheet_name= "Quarter Balance Sheet")

# Income Statement

## Quarterly IS

In [24]:
income_state = company.get_income_stmt(freq="quarterly")
print(income_state.columns)
income_state.info()

DatetimeIndex(['2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30',
               '2024-03-31', '2023-12-31'],
              dtype='datetime64[ns]', freq=None)
<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, TaxEffectOfUnusualItems to OperatingRevenue
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   2025-03-31 00:00:00  47 non-null     object
 1   2024-12-31 00:00:00  51 non-null     object
 2   2024-09-30 00:00:00  49 non-null     object
 3   2024-06-30 00:00:00  50 non-null     object
 4   2024-03-31 00:00:00  51 non-null     object
 5   2023-12-31 00:00:00  7 non-null      object
dtypes: object(6)
memory usage: 2.8+ KB


In [25]:
income_state.head()

,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
TaxEffectOfUnusualItems,-23200000.0,0.0,-3840000.0,-7920000.0,-8640000.0,NaN
TaxRateForCalcs,0.4,0.0,0.24,0.24,0.24,NaN
NormalizedEBITDA,127000000.0,125000000.0,74000000.0,47000000.0,26000000.0,NaN
TotalUnusualItems,-58000000.0,-91000000.0,-16000000.0,-33000000.0,-36000000.0,NaN
TotalUnusualItemsExcludingGoodwill,-58000000.0,-91000000.0,-16000000.0,-33000000.0,-36000000.0,NaN


In [26]:
income_statement_extended = preprocess_columns(income_state, "Income Statement", 20)

Se hace
DatetimeIndex(['2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30',
               '2024-03-31'],
              dtype='datetime64[ns]', freq=None)
Index([            'index', 2025-03-31 00:00:00, 2024-12-31 00:00:00,
       2024-09-30 00:00:00, 2024-06-30 00:00:00, 2024-03-31 00:00:00],
      dtype='object')
RangeIndex(start=0, stop=51, step=1)
Cogemos index


In [27]:
income_statement_extended

,Level 2 Order,index,Date,Value,Year,Month,Level 0
0,1,OperatingRevenue,2025-03-31,773000000.0,2025,3,Income Statement
1,2,TotalRevenue,2025-03-31,773000000.0,2025,3,Income Statement
2,3,CostOfRevenue,2025-03-31,449000000.0,2025,3,Income Statement
3,4,GrossProfit,2025-03-31,324000000.0,2025,3,Income Statement
4,5,OtherGandA,2025-03-31,113000000.0,2025,3,Income Statement
...,...,...,...,...,...,...,...
250,47,TotalUnusualItemsExcludingGoodwill,2024-03-31,-36000000.0,2024,3,Income Statement
251,48,TotalUnusualItems,2024-03-31,-36000000.0,2024,3,Income Statement
252,49,NormalizedEBITDA,2024-03-31,26000000.0,2024,3,Income Statement
253,50,TaxRateForCalcs,2024-03-31,0.24,2024,3,Income Statement


## Create Level 1

In [ ]:
total = ["Gross profit", "Total Non Current Assets", "Total Assets", "Current Liabilities",
          "Total Non Current Liabilities Net Minority Interest", "Total Liabilities Net Minority Interest",
            ]

In [47]:
levels_1_balance =  last_of_each_level = [
    "GrossProfit",                               
        "OperatingExpense",                          
        "OperatingIncome",                           
        "NetNonOperatingInterestIncomeExpense",      
        "GainOnSaleOfSecurity",                      
        "EarningsFromEquityInterest",                
        "SpecialIncomeCharges",                      
        "OtherNonOperatingIncomeExpenses",           
        "OtherIncomeExpense",                        
        "PretaxIncome",                              
    
    "TaxProvision",                              
    
    "NetIncome",           
    
    "TotalOperatingIncomeAsReported",            
    
    "TotalExpenses",                             
    
    "NormalizedIncome",                          
    
    "NetInterestIncome",                         
    "EBIT",                                      
    
    "EBITDA",                                    
    "ReconciledCostOfRevenue",                   
    "ReconciledDepreciation",                    
    
    "NetIncomeFromContinuingOperationNetMinorityInterest",  
    "TotalUnusualItems",                         
    "NormalizedEBITDA",                          
    "TaxEffectOfUnusualItems"                    
]

In [48]:

income_statement_level1 = create_level(income_statement_extended, levels_1_balance, levels_1_balance)

Pasamos a OperatingExpense
Pasamos a OperatingIncome
Pasamos a NetNonOperatingInterestIncomeExpense
Pasamos a GainOnSaleOfSecurity
Pasamos a EarningsFromEquityInterest
Pasamos a SpecialIncomeCharges
Pasamos a OtherNonOperatingIncomeExpenses
Pasamos a OtherIncomeExpense
Pasamos a PretaxIncome
Pasamos a TaxProvision
Pasamos a NetIncome
Pasamos a TotalOperatingIncomeAsReported
Pasamos a TotalExpenses
Pasamos a NormalizedIncome
Pasamos a NetInterestIncome
Pasamos a EBIT
Pasamos a EBITDA
Pasamos a ReconciledCostOfRevenue
Pasamos a ReconciledDepreciation
Pasamos a NetIncomeFromContinuingOperationNetMinorityInterest
Pasamos a TotalUnusualItems
Pasamos a NormalizedEBITDA
Pasamos a TaxEffectOfUnusualItems
Pasamos a GrossProfit
Pasamos a OperatingExpense
Pasamos a OperatingIncome
Pasamos a NetNonOperatingInterestIncomeExpense
Pasamos a GainOnSaleOfSecurity
Pasamos a EarningsFromEquityInterest
Pasamos a SpecialIncomeCharges
Pasamos a OtherNonOperatingIncomeExpenses
Pasamos a OtherIncomeExpense
Pa

## Create Level 2

In [53]:
highlighted_items = [
        "GrossProfit",
        "OperatingIncome",
        "NetNonOperatingInterestIncomeExpense",
        "GainOnSaleOfSecurity",
        "EarningsFromEquityInterest",
        "SpecialIncomeCharges",
        "OtherNonOperatingIncomeExpenses",
        "OtherIncomeExpense",
        "PretaxIncome",
        "TaxProvision",
        "NetIncome",
        "DilutedNIAvailtoComStockholders",
        "TotalOperatingIncomeAsReported",
        "TotalExpenses",
        "NormalizedIncome",
        "EBIT",
        "EBITDA",
        "ReconciledDepreciation",
        "NetIncomeFromContinuingOperationNetMinorityInterest",
        "TotalUnusualItems",
        "NormalizedEBITDA",
        "TaxEffectOfUnusualItems"
    ]


income_statement_level1_2 = create_level(income_statement_level1, highlighted_items, highlighted_items, level_str="Level 2", total=total)

Pasamos a OperatingIncome
Pasamos a NetNonOperatingInterestIncomeExpense
Pasamos a GainOnSaleOfSecurity
Pasamos a EarningsFromEquityInterest
Pasamos a SpecialIncomeCharges
Pasamos a OtherNonOperatingIncomeExpenses
Pasamos a OtherIncomeExpense
Pasamos a PretaxIncome
Pasamos a TaxProvision
Pasamos a NetIncome
Pasamos a DilutedNIAvailtoComStockholders
Pasamos a TotalOperatingIncomeAsReported
Pasamos a TotalExpenses
Pasamos a NormalizedIncome
Pasamos a EBIT
Pasamos a EBITDA
Pasamos a ReconciledDepreciation
Pasamos a NetIncomeFromContinuingOperationNetMinorityInterest
Pasamos a TotalUnusualItems
Pasamos a NormalizedEBITDA
Pasamos a TaxEffectOfUnusualItems
Pasamos a GrossProfit
Pasamos a OperatingIncome
Pasamos a NetNonOperatingInterestIncomeExpense
Pasamos a GainOnSaleOfSecurity
Pasamos a EarningsFromEquityInterest
Pasamos a SpecialIncomeCharges
Pasamos a OtherNonOperatingIncomeExpenses
Pasamos a OtherIncomeExpense
Pasamos a PretaxIncome
Pasamos a TaxProvision
Pasamos a NetIncome
Pasamos a 

## Exportamos IS

In [54]:
income_statement_level1_2.to_excel(f"C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/income_statement.xlsx", sheet_name= "Income Statement")

# Cash Flow Statement

In [ ]:
cash_flow_state = company.get_cash_flow(freq="quarterly")

In [ ]:
cash_flow_state.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, FreeCashFlow to NetIncomeFromContinuingOperations
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   2025-03-31 00:00:00  57 non-null     object
 1   2024-12-31 00:00:00  56 non-null     object
 2   2024-09-30 00:00:00  57 non-null     object
 3   2024-06-30 00:00:00  56 non-null     object
 4   2024-03-31 00:00:00  58 non-null     object
 5   2023-12-31 00:00:00  3 non-null      object
dtypes: object(6)
memory usage: 3.2+ KB


In [ ]:
cash_flow_state.head()

,2025-03-31,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
FreeCashFlow,47000000.0,209000000.0,302000000.0,253000000.0,-25000000.0,NaN
RepurchaseOfCapitalStock,0.0,-37000000.0,-58000000.0,-34000000.0,-97000000.0,NaN
RepaymentOfDebt,-56000000.0,-52000000.0,-50000000.0,-49000000.0,-531000000.0,NaN
IssuanceOfDebt,32000000.0,26000000.0,37000000.0,27000000.0,30000000.0,NaN
CapitalExpenditure,-26000000.0,-44000000.0,-36000000.0,-19000000.0,-14000000.0,NaN


In [ ]:
cash_flow_state_extended = preprocess_columns(cash_flow_state, "Cash Flow Statement", 20)

Se hace
DatetimeIndex(['2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30',
               '2024-03-31'],
              dtype='datetime64[ns]', freq=None)
Index([            'index', 2025-03-31 00:00:00, 2024-12-31 00:00:00,
       2024-09-30 00:00:00, 2024-06-30 00:00:00, 2024-03-31 00:00:00],
      dtype='object')
RangeIndex(start=0, stop=59, step=1)
Cogemos index


In [ ]:
cash_flow_state_extended.head()

,Level 2 Order,index,Date,Value,Year,Month,Level 0
0,1,NetIncomeFromContinuingOperations,2025-03-31,24000000.0,2025,3,Cash Flow Statement
1,2,GainLossOnSaleOfPPE,2025-03-31,-1000000.0,2025,3,Cash Flow Statement
2,3,EarningsLossesFromEquityInvestments,2025-03-31,-1000000.0,2025,3,Cash Flow Statement
3,4,OperatingGainsLosses,2025-03-31,-2000000.0,2025,3,Cash Flow Statement
4,5,Depreciation,2025-03-31,32000000.0,2025,3,Cash Flow Statement


In [ ]:
levels_1_cashflow =  ["OperatingCashFlow", "InvestingCashFlow", "FinancingCashFlow", "EndCashPosition", 
                     "RepurchaseOfCapitalStock", "FreeCashFlow"]
name_levels_1_cashflow = ["CFO", "CFI", "CFF", "Cash", "Other", "FCFF"]

cash_flow_level1 = create_level(cash_flow_state_extended, levels_1_cashflow, 
                                          name_levels_1_cashflow)

Pasamos a InvestingCashFlow
Pasamos a FinancingCashFlow
Pasamos a EndCashPosition
Pasamos a RepurchaseOfCapitalStock
Pasamos a FreeCashFlow
Pasamos a OperatingCashFlow
Pasamos a InvestingCashFlow
Pasamos a FinancingCashFlow
Pasamos a EndCashPosition
Pasamos a RepurchaseOfCapitalStock
Pasamos a FreeCashFlow
Pasamos a OperatingCashFlow
Pasamos a InvestingCashFlow
Pasamos a FinancingCashFlow
Pasamos a EndCashPosition
Pasamos a RepurchaseOfCapitalStock
Pasamos a FreeCashFlow
Pasamos a OperatingCashFlow
Pasamos a InvestingCashFlow
Pasamos a FinancingCashFlow
Pasamos a EndCashPosition
Pasamos a RepurchaseOfCapitalStock
Pasamos a FreeCashFlow
Pasamos a OperatingCashFlow
Pasamos a InvestingCashFlow
Pasamos a FinancingCashFlow
Pasamos a EndCashPosition
Pasamos a RepurchaseOfCapitalStock
Pasamos a FreeCashFlow
Pasamos a OperatingCashFlow


## Exportamos CS

In [ ]:
cash_flow_level1.to_excel(f"C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/cash_flow_statement.xlsx", sheet_name= "Cash Flow Statement")

# Competition

## Sector 1

In [ ]:
sector_1 = "Competitors"

### Tickers

In [ ]:
competitors_1 = {
    "BYD": "BYDDY",
    "NIO": "NIO",
    "Xpeng": "XPEV",
    "Li_Auto": "LI",
    "Rivian": "RIVN",
    "Lucid": "LCID",
    "Ford": "F",
    "General_Motors": "GM"
}

In [ ]:
competitors_1 = {
    "Uber": "UBER",           # Competidor global de ride-hailing :contentReference[oaicite:1]{index=1}
    "Lyft": "LYFT",           # Competidor en EE. UU. :contentReference[oaicite:2]{index=2}
    "Ryde": "RYDE",           # Cotiza en NYSE desde marzo 2024 :contentReference[oaicite:6]{index=6}
}

special_mention = ["Cabify", "Didi", "inDrive", "Ola"]

## Sector 2

In [ ]:
sector_2 = " Energy and batteries"

### Tickers

In [ ]:
competitors_2 = {
    "Enphase_Energy": "ENPH",
    "SolarEdge": "SEDG",
    "SunPower": "SPWR",
    "Fluence_Energy": "FLNC",
    "NextEra_Energy": "NEE"
}

## Sector 3

In [ ]:
sector_3 = "Technology"

### Tickers

In [ ]:
competitors_3 = {
    "Waymo_Alphabet": "GOOGL",
    "Cruise_GM": "GM", 
    "Mobileye": "MBLY",
    "Aurora": "AUR",
    "Apple": "AAPL"
}

## Sector 4

In [ ]:
sector_4 = "Substitutes Vehicles"

### Tickers

In [ ]:
competitors_4 = {
    "Toyota": "TM",
    "Honda": "HMC",
    "Nissan": "NSANY",
    "Renault": "RNO",
    "Chevrolet": "GM",      
    "Mercedes_Benz": "MBGYY",
    "BMW": "BMWYY",
    "Audi": "NSU",             
    "Porsche": "POAHY",
    "Volvo": "VOLVY"
}


# Historical Stock Price

In [ ]:
years_data='15Y'

In [ ]:
company_historical = company.history(period=years_data)

In [ ]:
company_historical.reset_index(inplace=True)

In [ ]:
company_historical

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-12-01 00:00:00-05:00,11.890,11.890,11.89,11.89,500,0.0,0.0
1,2020-12-02 00:00:00-05:00,12.480,12.480,11.82,11.82,1000,0.0,0.0
2,2020-12-03 00:00:00-05:00,11.820,11.900,11.80,11.90,14100,0.0,0.0
3,2020-12-04 00:00:00-05:00,12.990,13.980,11.50,12.40,15500,0.0,0.0
4,2020-12-07 00:00:00-05:00,13.150,14.000,12.20,12.55,14500,0.0,0.0
...,...,...,...,...,...,...,...,...
1148,2025-06-30 00:00:00-04:00,4.985,5.120,4.86,5.03,53350300,0.0,0.0
1149,2025-07-01 00:00:00-04:00,5.040,5.065,4.86,4.93,30189600,0.0,0.0
1150,2025-07-02 00:00:00-04:00,4.930,5.180,4.87,4.90,52767700,0.0,0.0
1151,2025-07-03 00:00:00-04:00,4.920,4.950,4.83,4.87,19438400,0.0,0.0


In [ ]:
company_historical.dtypes

Date            datetime64[ns, America/New_York]
Open                                     float64
High                                     float64
Low                                      float64
Close                                    float64
Volume                                     int64
Dividends                                float64
Stock Splits                             float64
dtype: object

In [ ]:
company_historical['Date'] = pd.to_datetime(company_historical['Date']).dt.date


In [ ]:
company_historical

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-12-01,11.890,11.890,11.89,11.89,500,0.0,0.0
1,2020-12-02,12.480,12.480,11.82,11.82,1000,0.0,0.0
2,2020-12-03,11.820,11.900,11.80,11.90,14100,0.0,0.0
3,2020-12-04,12.990,13.980,11.50,12.40,15500,0.0,0.0
4,2020-12-07,13.150,14.000,12.20,12.55,14500,0.0,0.0
...,...,...,...,...,...,...,...,...
1148,2025-06-30,4.985,5.120,4.86,5.03,53350300,0.0,0.0
1149,2025-07-01,5.040,5.065,4.86,4.93,30189600,0.0,0.0
1150,2025-07-02,4.930,5.180,4.87,4.90,52767700,0.0,0.0
1151,2025-07-03,4.920,4.950,4.83,4.87,19438400,0.0,0.0


In [ ]:
company_historical.dtypes

Date             object
Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object

In [ ]:
columns = ['Open', 'High', 'Low', 'Close']
company_historical[columns] = company_historical[columns].apply(lambda x: round(x,3))

In [ ]:
company_historical.to_excel("C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/company_stock_price.xlsx", sheet_name= company_name)

# Preprocessing historicals of competition

In [ ]:
def preprocess_historical_data(df):
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    columns = ['Open', 'High', 'Low', 'Close']
    df[columns] = df[columns].apply(lambda x: round(x,3))

    return df

## Sector 1

In [ ]:
competitors_1_dict = {}

for competitor in competitors_1:
    
    competitor_ticker = yf.Ticker(competitor)
    competitors_1_dict[competitor] = preprocess_historical_data(competitor_ticker.history(period=years_data))
    print(f"{competitor}")

Uber
Lyft
Ryde


In [ ]:
competitors_1_dict

{'Uber':             Date    Open    High    Low  Close     Volume  Dividends  \
 0     2019-05-10  42.000  45.000  41.06  41.57  186322500        0.0   
 1     2019-05-13  38.790  39.240  36.08  37.10   79442400        0.0   
 2     2019-05-14  38.310  39.960  36.85  39.96   46661100        0.0   
 3     2019-05-15  39.370  41.880  38.95  41.29   36086100        0.0   
 4     2019-05-16  41.480  44.060  41.25  43.00   38115500        0.0   
 ...          ...     ...     ...    ...    ...        ...        ...   
 1542  2025-06-30  91.685  93.385  91.52  93.30   16155300        0.0   
 1543  2025-07-01  92.850  93.240  90.31  92.13   17124000        0.0   
 1544  2025-07-02  91.880  94.100  91.55  92.05   13604000        0.0   
 1545  2025-07-03  92.630  94.110  92.61  93.63    8112600        0.0   
 1546  2025-07-07  93.540  97.120  93.54  96.68   29785600        0.0   
 
       Stock Splits  
 0              0.0  
 1              0.0  
 2              0.0  
 3              0.0  
 4  

In [ ]:
first_competitor = next(iter(competitors_1_dict.values()))
first_competitor

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-05-10,42.000,45.000,41.06,41.57,186322500,0.0,0.0
1,2019-05-13,38.790,39.240,36.08,37.10,79442400,0.0,0.0
2,2019-05-14,38.310,39.960,36.85,39.96,46661100,0.0,0.0
3,2019-05-15,39.370,41.880,38.95,41.29,36086100,0.0,0.0
4,2019-05-16,41.480,44.060,41.25,43.00,38115500,0.0,0.0
...,...,...,...,...,...,...,...,...
1542,2025-06-30,91.685,93.385,91.52,93.30,16155300,0.0,0.0
1543,2025-07-01,92.850,93.240,90.31,92.13,17124000,0.0,0.0
1544,2025-07-02,91.880,94.100,91.55,92.05,13604000,0.0,0.0
1545,2025-07-03,92.630,94.110,92.61,93.63,8112600,0.0,0.0


In [ ]:
with pd.ExcelWriter('C:/Users/jaime/Escritorio/Github/X_Financial_Analyst/competitors_1.xlsx', engine='openpyxl') as writer:
    for df_name, df in competitors_1_dict.items():
        df.to_excel(writer, sheet_name=df_name)

## Sector 2

## Sector 3

## Sector 4

# Algorithm

With Bollinger Bands, analyze the company stock price and try to detect anomalies. Then compare with the competitors

# Sentiment Analysis

Use the Ollama model to analyze and to extract the news and see the actual sentiment with the company

# Portfolio Advisor

How this company can be included in a portfolio.